In [237]:
import math
import random

def generate_data(num_trials):
    trials = []
    for _ in range(num_trials):
        count = 0
        while True:
            count += 1
            if random.randint(1, 125) == 1:
                break
        trials.append(count)
    return trials

x_data = generate_data(200)

def apply_formula(x):
    y = round(1-math.exp(-x/125),4)
    return y

def generate_y_list(x_list):
    y_list = []
    for x in x_list:
        y = apply_formula(x)
        y_list.append(y)
    return y_list

# Y 데이터 리스트 생성
y_data = generate_y_list(x_data)

print("X 데이터:", x_data)
#print("Y 데이터:", y_data)

X 데이터: [76, 54, 34, 29, 81, 40, 492, 222, 265, 182, 105, 6, 35, 524, 116, 329, 31, 99, 49, 78, 241, 96, 355, 87, 67, 184, 335, 435, 160, 318, 520, 143, 25, 10, 64, 23, 7, 91, 34, 713, 8, 33, 146, 32, 9, 216, 9, 118, 184, 258, 91, 35, 21, 31, 44, 49, 83, 35, 138, 83, 63, 317, 35, 20, 25, 150, 5, 47, 212, 43, 8, 32, 175, 22, 39, 83, 683, 63, 511, 89, 11, 82, 164, 554, 183, 121, 168, 48, 186, 329, 34, 267, 338, 39, 51, 16, 46, 81, 32, 228, 49, 149, 94, 142, 177, 98, 60, 53, 140, 29, 103, 55, 177, 1, 90, 154, 178, 24, 40, 85, 18, 14, 152, 57, 23, 49, 181, 156, 117, 21, 103, 17, 57, 192, 12, 19, 50, 35, 175, 36, 360, 134, 23, 84, 98, 20, 4, 210, 125, 93, 131, 277, 162, 14, 9, 152, 193, 72, 111, 178, 105, 18, 131, 46, 38, 8, 98, 106, 114, 14, 157, 306, 289, 3, 92, 502, 86, 71, 141, 6, 391, 44, 181, 1, 16, 5, 198, 5, 30, 10, 111, 310, 119, 155, 343, 130, 274, 82, 208, 412]


# LSTM 이용시도
10개씩 쪼개어서 그다음 나오는 숫자를 타겟으로 하여 학습시키는 것도 가능할것같다. 근데 이렇게하면 10개가 채워지기 전에는 예측을 해주지 못한다. 하지만 10개가 넘어가고 부터는 계속 예측가능하다.

### 시행횟수를 줄여 데이터의 크기를 작게하여 확인을 해보았는데 스탭크기가 8이면 처음부터 8개를 X로해서 y(lable)은 9번째수로 하여 맞게 데이터 세트가 만들어진다. lable값이 데이터세트 리스트의 맨 마지막 숫자가 되었을때 실행을 마친다.

In [238]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

# 데이터셋 생성 함수
def create_dataset(numbers, n_steps):
    X, y = [], []
    for i in range(len(numbers)):
        end_ix = i + n_steps
        if end_ix > len(numbers)-1:
            break
        seq_x, seq_y = numbers[i:end_ix], numbers[end_ix] # end_ix는 포함되지 않는다.
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# 숫자 시퀀스 정의
numbers = x_data
n_steps = 20 # 학습하고싶은 숫자묶음의 크기(발걸음)을 설정합니다.

# 데이터셋 생성
X, y = create_dataset(numbers, n_steps) #데이터셋 생성 함수 활성화

print(X)
#print(y)

[[ 76  54  34 ...  99  49  78]
 [ 54  34  29 ...  49  78 241]
 [ 34  29  81 ...  78 241  96]
 ...
 [ 71 141   6 ... 343 130 274]
 [141   6 391 ... 130 274  82]
 [  6 391  44 ... 274  82 208]]


In [233]:
# LSTM 모델 정의
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# 모델 훈련을 위해 X를 재구성
X = X.reshape((X.shape[0], X.shape[1], 1))

# 조기 종료 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 모델 훈련
model.fit(X, y, epochs=2000, verbose=1, callbacks=[early_stopping]) # 일부러 에포크를 매우 크게 하였습니다.

Epoch 1/2000
6/6 [==============================] - 1s 5ms/step - loss: 35978.5039
Epoch 2/2000
6/6 [==============================] - 0s 4ms/step - loss: 33395.8633
Epoch 3/2000
6/6 [==============================] - 0s 4ms/step - loss: 30486.8828
Epoch 4/2000
6/6 [==============================] - 0s 4ms/step - loss: 27645.6777
Epoch 5/2000
6/6 [==============================] - 0s 4ms/step - loss: 23442.5137
Epoch 6/2000
6/6 [==============================] - 0s 4ms/step - loss: 23686.9805
Epoch 7/2000
6/6 [==============================] - 0s 4ms/step - loss: 23184.1309
Epoch 8/2000
6/6 [==============================] - 0s 4ms/step - loss: 23364.5527
Epoch 9/2000
6/6 [==============================] - 0s 4ms/step - loss: 23100.6758
Epoch 10/2000
6/6 [==============================] - 0s 4ms/step - loss: 22351.3867
Epoch 11/2000
6/6 [==============================] - 0s 4ms/step - loss: 21679.1367
Epoch 12/2000
6/6 [==============================] - 0s 4ms/step - loss: 20894.3945
E

In [236]:
#이렇게 나눠놓으니 위에 학습한것을 토대로 계속 학습하면 시간 안걸리고 다음숫자 예측할수있다.

# 예측을 위한 새로운 데이터 포인트
x_input = np.array([36, 20, 365, 146, 20, 22, 163, 120, 23, 1, 20, 129, 97, 297, 224, 172,105, 93, 238, 143])
x_input = x_input.reshape((1, n_steps, 1))

# 다음 숫자 예측
yhat = model.predict(x_input, verbose=1) # verbose 설정에따라 학습상황을 아려줍니다.
yhat[0][0]  # 예측된 다음 숫자 출력

1/1 [==============================] - 0s 10ms/step


99.281204

# LSTM 숫자 우연인지 몇번 정확히 맞췃다.
데이터 세트의 크기를 키우며 어디서 맞추는지 실험해본다. 1.스텝의 크기 2.데이터세트의 크기 조절해볼것 세트 크기 (1000개:120예측 10000:90예측 2000:112예측) 1000개에서 더 근사하게 맞춘다.
손좀봐보자
정확히 맞춘것은 데이터세트 1000개 숫자배열 크기가 8일때였다
###같은 조건으로 반복하여도 예측숫자는 다르게 출력된다.
### 계속해보니  확률적으로 근사하여서 맞추는것같긴하다
우연이 아니었다 1000개 8개일때 적중하는 숫자있다. 어느정도 맞추는것같다 모델 조금더 손보자 그리고 (나중에 내가 모델 만들어보기도 하자)
for문으로 5번정도 시켜서 숫자보기
실험 (159 130 170 109 128)

# LSTM 두번째 모델

In [106]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 모델 정의
n_steps = 5  # 입력 시퀀스의 길이
n_features = 1  # 특징의 개수

model = Sequential([
    LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
    Dense(2)  # 두 개의 출력: 예측값과 불확실성
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 예측을 위한 새로운 데이터 포인트
x_input = np.array([23, 1, 20, 129, 97])
x_input = x_input.reshape((1, n_steps, 1))

# 예측
yhat = model.predict(x_input, verbose=0)  # verbose=0으로 설정하여 학습 상황 출력 없음
prediction, uncertainty = yhat[0][0], yhat[0][1]  # 예측된 다음 숫자와 불확실성 출력

print(f"예측된 값: {prediction}, 불확실성: {uncertainty}")
#불확실성은 낮을수록 좋다.(0에 가까울수록)

예측된 값: -17.43350601196289, 불확실성: -4.0912675857543945


- 램덤에 대한 예측을 시도해보았다. 완전한 램덤은 아니지만 램덤시행에서 파생되었기때문에 유사램덤이긴 하다. 우선 스텝의 크기 데이터의 크기 얼리스탑핑 페이션트 크기 등 여러가지를 조절해봒는데 랜덤을 학습하기에는 맞지않고 랜덤은 학습이 불가능한데 유사랜덤이라 매우 학습하기 어려웠던것같다.